In [2]:
import sklearn
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_selection import mutual_info_classif
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold 
from sklearn.feature_selection import mutual_info_classif

In [3]:
X_train = pd.read_csv('Train-Test Data/ant/without_ant_data.csv', index_col=0)
X_test = pd.read_csv('Train-Test Data/ant/Test/ant_test_1.csv', index_col=0)

In [4]:
Y_train = X_train["label"]
Y_test = X_test["label"]

In [5]:
X_train.drop(columns=["bugs","label"],inplace=True)
X_test.drop(columns=["bugs","label"],inplace=True)

In [6]:
X_train = X_train.iloc[:,3:]

In [7]:
X_test = X_test.iloc[:,3:]

In [8]:
X_test.iloc[:2,:]

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,...,ic,cbm,amc,nr,ndc,nml,isOld,ndpv,max(cc),avg(cc)
id,,,,,,,,,,,,,,,,,,,,,
1162,17,2,0.0,9,64,76,7,2.0,9,0.879167,...,1.0,3.0,40.058823,1.0,1.0,38.0,1.0,0.0,9,1.411765
1163,10,1,4.0,17,81,5,4,13.0,4,0.783626,...,0.0,0.0,71.400000,1.0,1.0,38.0,1.0,0.0,14,4.200000


In [9]:
X_test.iloc[:2,:] * (X_train.mean() / X_test.mean())

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,...,ic,cbm,amc,nr,ndc,nml,isOld,ndpv,max(cc),avg(cc)
id,,,,,,,,,,,,,,,,,,,,,
1162,11.679745,2.716786,0.000000,14.556357,56.274677,68.242644,4.770103,4.754386,5.338376,1.321184,...,1.522475,5.609931,48.878768,1.061628,0.653594,35.32959,0.927688,NaN,8.515607,1.462027
1163,6.870438,1.358393,4.846342,27.495341,71.222638,4.489648,2.725773,30.903507,2.372612,1.177608,...,0.000000,0.000000,87.120483,1.061628,0.653594,35.32959,0.927688,NaN,13.246499,4.349532


In [10]:
X_test.mean(), X_train.mean()

(wmc          8.278075
 dit          2.053476
 noc          0.443850
 cbo          8.385027
 rfc         26.636364
 lcom        46.951872
 ca           4.390374
 ce           4.433155
 npm          6.540107
 lcom3        0.849310
 loc        223.593583
 dam          0.772852
 moa          0.743316
 mfa          0.408395
 cam          0.514752
 ic           0.625668
 cbm          0.844920
 amc         23.404408
 nr           1.000000
 ndc          1.000000
 nml         38.000000
 isOld        1.000000
 ndpv         0.000000
 max(cc)      3.438503
 avg(cc)      1.215217
 dtype: float64,
 wmc          5.687400
 dit          2.789427
 noc          0.537763
 cbo         13.561716
 rfc         23.421137
 lcom        42.159472
 ca           2.991791
 ce          10.538464
 npm          3.879283
 lcom3        1.276317
 loc        173.396510
 dam          0.245946
 moa          0.256909
 mfa          0.565577
 cam          0.552687
 ic           0.952565
 cbm          1.579981
 amc         28.5

## Watanabe

In [11]:
def watanabe_transform(X_train, X_test):
    mean_train = X_train.mean()
    mean_test = X_test.mean()
    X_test_transformed = X_test * (mean_train / mean_test)
    return X_test_transformed

# Apply the transformation
X_test_transformed = watanabe_transform(X_train, X_test)

## 2009Cruz Method

In [12]:
def cruz_transform(X_train, X_test):
    median_train = X_train.median()
    median_test = X_test.median()
    X_test_transformed = X_test + (median_train - median_test)
    return X_test_transformed

## 2009Turhan Method

In [40]:
knn = NearestNeighbors(n_neighbors=10, metric='euclidean')
knn.fit(X_train)

NearestNeighbors(metric='euclidean', n_neighbors=10)

In [56]:
distances, indices = knn.kneighbors(X_test)

In [58]:
len(indices.flatten())

1870

In [59]:
len(np.unique(indices.flatten()))

1622

In [ ]:
#X_train.iloc[np.unique(indices.flatten())].duplicates()

In [45]:
len(indices.flatten())

1870

In [ ]:
def knn_filter(X_train, X_test, k=10):
    knn = NearestNeighbors(n_neighbors=k, metric='euclidean')
    knn.fit(X_train)
    distances, indices = knn.kneighbors(X_test)
    # To get the filtered training set without duplicates
    #X_train_filtered = X_train[np.unique(indices.flatten())]
    #X_train_filtered = np.unique(X_train[indices.flatten()], axis=0)

    X_train_filtered = X_train.iloc[np.unique(indices.flatten())]
    return X_train_filtered

## Split by Projects

In [11]:
X_train_org = pd.read_csv('Train-Test Data/ant/without_ant_data.csv', index_col=0)
X_test_org = pd.read_csv('Train-Test Data/ant/Test/ant_test_1.csv', index_col=0)

In [12]:
def get_project_indices(df, project_column='Project'):
    # Group the dataframe by the project column and get the indices
    grouped = df.groupby(project_column)
    project_indices = {name: group.index for name, group in grouped}
    return project_indices

def split_dataframe_by_project_indices(df, project_indices):
    # Create a dictionary to store each project's dataframe
    project_dfs = [{'name':name,'df':df.loc[indices]} for name, indices in project_indices.items()]
    return project_dfs

In [13]:
X_train_indices = get_project_indices(X_train_org)
X_train_ps = split_dataframe_by_project_indices(X_train,X_train_indices)

In [29]:
X_train_ps[0]

{'name': 'Tomcat',
 'df':       wmc  dit  noc  cbo  rfc  lcom  ca    ce  npm     lcom3  ...   ic  cbm  \
 id                                                            ...             
 0      14    1  0.0    1   15    91   0  10.0   14  2.000000  ...  0.0  0.0   
 1      11    2  0.0    0   46    39   0  10.0    8  0.550000  ...  1.0  2.0   
 2      37    1  1.0    8   68     0   7  10.0   36  0.912835  ...  0.0  0.0   
 3      14    5  0.0    2   34    29   0  10.0   14  0.769231  ...  1.0  1.0   
 4       4    1  0.0    2   16     4   0  10.0    2  0.666667  ...  0.0  0.0   
 ...   ...  ...  ...  ...  ...   ...  ..   ...  ...       ...  ...  ...  ...   
 1157    2    2  0.0    2    4     1   0  10.0    1  1.000000  ...  0.0  0.0   
 1158    1    1  0.0    0    2     0   0  10.0    1  2.000000  ...  0.0  0.0   
 1159    3    3  0.0    2    7     0   2  10.0    3  0.000000  ...  2.0  2.0   
 1160    1    1  0.0    2    2     0   1  10.0    0  2.000000  ...  0.0  0.0   
 1161    9    1

In [14]:
Y_train_ps = split_dataframe_by_project_indices(Y_train,X_train_indices)

# 2013He

In [43]:
def he_method(X_train_projects, X_test, y_train_projects,classifiers=[RandomForestClassifier], N=10, FSS=0.8):
    distances = []
    SAMS=[]
    # Step 1: Calculate distances between each training set and the test set
    for i in range(len(X_train_projects)):
        p_df = X_train_projects[i]['df']
        p_name = X_train_projects[i]['name']
        print(f'Processing {p_name} Project')
        K = min(500,len(p_df), len(X_test))
        SAMtrain = p_df.sample(n=K, random_state=42)
        SAMtest = X_test.sample(n=K, random_state=42)
        
        SAMtrain['label'] = 1
        SAMtest['label'] = -1
        
        SAM = pd.concat([SAMtrain, SAMtest])
        SAMS.append(SAM)
        y_SAM = SAM['label']
        X_SAM = SAM.drop('label', axis=1)
        kf = KFold(n_splits=5)
        classifier = LogisticRegression(random_state=42)
        classifier.fit(X_SAM, y_SAM)
        
        accuracy = accuracy_score(y_SAM, classifier.predict(X_SAM))
        distances.append(2 * (accuracy - 0.5))  # Use 1 - accuracy as distance measure
    # Step 2: Select top_N training sets with lowest distance
    top_N_indices = np.argsort(distances)[:N]
    selected_train_sets = [X_train_projects[i]['df'] for i in top_N_indices]
    selected_train_labels = [y_train_projects[i]['df'] for i in top_N_indices]

    # Step 3: Remove unstable features using Information Gain
    def remove_unstable_features(X, y, ratio):
        # Calculate information gain for each feature
        info_gains = mutual_info_classif(X, y)
        
        # Sort features by information gain (descending order)
        sorted_features = pd.Series(info_gains, index=X.columns).sort_values(ascending=False)
        
        # Select stable features (those with lower information gain)
        n_keep = int(len(sorted_features) * (1 - ratio))
        stable_features = sorted_features.index[-n_keep:]
        
        return X[stable_features]
    
    # Apply feature selection to each training set (maybe SAMS should be passed here instead, not sure)
    selected_train_sets = [remove_unstable_features(X, y, FSS) 
                           for X, y in zip(selected_train_sets, selected_train_labels)]
    
    # Apply feature selection to test set (using the first training set for consistency)
    #X_test_stable = remove_unstable_features(X_test, y_train_projects[top_N_indices[0]]['df'], FSS)
    # Step 4: Learn prediction models and ensemble
    cls_predictions=[]
    for cls in classifiers:
        print(f'Predicting with {cls} classifier..')
        predictions = []
        for X_train, y_train in zip(selected_train_sets, selected_train_labels):
            model = cls(random_state=42)
            model.fit(X_train, y_train)
            # Use same features as X_train
            X_test_stable = X_test[X_train.columns] 
            pred = model.predict_proba(X_test_stable)[:, 1]
            predictions.append(pred)
    
        # Calculate average prediction scores
        final_predictions = np.mean(predictions, axis=0)
        cls_predictions.append({'model':type(cls).__name__,'final_predictions':final_predictions})
        
    return cls_predictions

In [44]:
res = he_method(X_train_ps,X_test,Y_train_ps)

Processing Tomcat Project
Processing camel Project
Processing ivy Project
Processing jEdit Project


d:\Miniconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Miniconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Processing log4j Project
Processing lucene Project
Processing poi Project
Processing prop-1 Project


d:\Miniconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Miniconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Processing prop-2 Project
Processing prop-3 Project
Processing prop-4 Project
Processing prop-42 Project


...

Processing student-25 Project
Processing student-26 Project
Processing student-27 Project
Processing student-28 Project
Processing student-29 Project
Processing student-30 Project
Processing student-31 Project
Processing student-33 Project
Processing student-34 Project
Processing student-36 Project
Processing student-37 Project


d:\Miniconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
d:\Miniconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_resu

Processing synapse Project
Processing velocity Project
Processing xalan Project
Processing xerces Project


d:\Miniconda\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Predicting with <class 'sklearn.ensemble._forest.RandomForestClassifier'> classifier..


In [45]:
res

[{'model': 'ABCMeta',
  'final_predictions': array([0.32379278, 0.57858699, 0.25366328, 0.25585542, 0.33963062,
         0.36969592, 0.25227442, 0.31408896, 0.28682399, 0.34337324,
         0.24242133, 0.24509822, 0.32744488, 0.22323268, 0.27199592,
         0.18646238, 0.18646238, 0.38423247, 0.30182525, 0.36245736,
         0.33760541, 0.33377156, 0.31040806, 0.25651366, 0.38962817,
         0.22323268, 0.35542758, 0.18646238, 0.28929078, 0.26253494,
         0.2347456 , 0.2258312 , 0.35180678, 0.33974397, 0.22813567,
         0.18646238, 0.23396153, 0.18646238, 0.24407115, 0.2869165 ,
         0.29130577, 0.24487681, 0.3116602 , 0.22323268, 0.22323268,
         0.36243003, 0.19727333, 0.25737324, 0.27772941, 0.31731127,
         0.19804369, 0.33395387, 0.23386888, 0.20088694, 0.35458144,
         0.25935151, 0.32352891, 0.29191575, 0.32424809, 0.32478496,
         0.28004313, 0.30740806, 0.22763883, 0.2127899 , 0.18646238,
         0.32151989, 0.31408896, 0.37262156, 0.36205067, 0.2

# 2013Herbold

In [33]:
def herbold_method(X_train_projects, X_test, k=0.5):
    distances = []
    SAMS=[]
    X_test_mean = X_test.mean(axis=0)
    X_test_std = X_test.std(axis=0)
    distances = []
    # Step 1: Calculate distances between each training set and the test set.
    for i in range(len(X_train_projects)):
        p_df = X_train_projects[i]['df']
        p_name = X_train_projects[i]['name']

        project_means = p_df.mean(axis=0)
        project_stds = p_df.std(axis=0)

        sim = np.sqrt(((project_means - X_test_mean) ** 2).mean() + ((project_stds - X_test_std) ** 2).mean())
        print(sim)
        distances.append(sim)
    #most_similar_projects = similarities.nsmallest(int(len(X_train) * k))
    indices = np.argsort(distances)
    #print(indices)
    selected_projects = [X_train_projects[i] for i in indices]
    #print([s['name'] for s in selected_projects])
    X_train_filtered = pd.concat([s['df'] for s in selected_projects])
    return selected_projects,X_train_filtered

In [34]:
len(X_train_ps)

35

In [35]:
res = herbold_method(X_train_ps,X_test,k=0.5)

170.07408804426166
53.51879129560863
126.66821726007726
266.8481203830648
24.294863298528586
82.98386903002074
59.90566408966359
179.1907138788903
22.55029476799095
23.515060878339014
31.805845282368146
45.96761735152414
33.74899621803404
34.04973749430619
51.9996795481187
23.396509519395863
42.34345165813229
22.092884395985426
89.9217309179522
66.41092099130306
73.88390881902929
45.78820122639203
43.05038637068302
67.70078091482017
69.05434206954294
33.868292504097205
77.20572054787986
64.83879465169207
41.71321348928689
27.279511917687998
37.74323397546259
37.24795289610897
138.7183276031552
116.88270046247058
170.77822238321446
[17  8 15  9  4 29 10 12 25 13 31 30 28 16 22 21 11 14  1  6 27 19 23 24
 20 26  5 18 33  2 32  0 34  7  3]
['student-22', 'prop-2', 'student-20', 'prop-3', 'log4j', 'student-36', 'prop-4', 'prop-43', 'student-30', 'prop-5', 'synapse', 'student-37', 'student-34', 'student-21', 'student-27', 'student-26', 'prop-42', 'student-18', 'camel', 'poi', 'student-33', 

In [39]:
res[0]

[{'name': 'student-22',
  'df':         wmc  dit  noc  cbo  rfc  lcom  ca    ce  npm     lcom3  ...   ic  cbm  \
  id                                                              ...             
  102235    3    2  0.0    8   19     3   1   7.0    3  2.000000  ...  1.0  1.0   
  102236    4    1  0.0    2    5     6   0   2.0    4  2.000000  ...  0.0  0.0   
  102237    2    2  0.0    1    4     1   1   0.0    1  2.000000  ...  0.0  0.0   
  102238    6    1  0.0    1    7    15   1   0.0    5  2.000000  ...  0.0  0.0   
  102239   10    6  0.0    6   50    33   4   5.0    9  0.916667  ...  0.0  0.0   
  102240    2    2  0.0    5   15     1   1   4.0    1  2.000000  ...  1.0  1.0   
  102241   12    5  5.0    8   13    64   7   1.0   11  0.909091  ...  0.0  0.0   
  102242    3    1  0.0    4    3     3   4   1.0    3  2.000000  ...  0.0  0.0   
  102243    2    2  0.0    3    8     1   1   2.0    1  2.000000  ...  0.0  0.0   
  102244    2    1  0.0    3    8     0   1   3.0    1  0

# 2012Ma

In [63]:
print(X_test.min())
print(X_test.max())

wmc         0.0
dit         1.0
noc         0.0
cbo         0.0
rfc         0.0
lcom        0.0
ca          0.0
ce          0.0
npm         0.0
lcom3       0.0
loc         0.0
dam         0.0
moa         0.0
mfa         0.0
cam         0.0
ic          0.0
cbm         0.0
amc         0.0
nr          1.0
ndc         1.0
nml        38.0
isOld       1.0
ndpv        0.0
max(cc)     0.0
avg(cc)     0.0
dtype: float64
wmc          71.000000
dit           6.000000
noc          34.000000
cbo         103.000000
rfc         186.000000
lcom       2247.000000
ca           99.000000
ce           21.000000
npm          59.000000
lcom3         2.000000
loc        2193.000000
dam           1.000000
moa           7.000000
mfa           1.000000
cam           1.000000
ic            4.000000
cbm           7.000000
amc         101.000000
nr            1.000000
ndc           1.000000
nml          38.000000
isOld         1.000000
ndpv          0.000000
max(cc)      35.000000
avg(cc)       5.869565
dtype: flo

In [64]:
for i, row in X_train.iterrows():
    h = ((X_test.min() <= row) & (row <= X_test.max())).astype(int)
    print(row)
    print(h)
    si = h.sum()
    weight = 1 / ((len(row) - si + 1) ** 2)
    break

wmc        14.000000
dit         1.000000
noc         0.000000
cbo         1.000000
rfc        15.000000
lcom       91.000000
ca          0.000000
ce         10.000000
npm        14.000000
lcom3       2.000000
loc        17.000000
dam         0.000000
moa         0.000000
mfa         0.000000
cam         0.328571
ic          0.000000
cbm         0.000000
amc         0.214286
nr          1.000000
ndc         1.000000
nml        38.000000
isOld       1.000000
ndpv        0.000000
max(cc)     1.000000
avg(cc)     0.928571
Name: 0, dtype: float64
wmc        1
dit        1
noc        1
cbo        1
rfc        1
lcom       1
ca         1
ce         1
npm        1
lcom3      1
loc        1
dam        1
moa        1
mfa        1
cam        1
ic         1
cbm        1
amc        1
nr         1
ndc        1
nml        1
isOld      1
ndpv       1
max(cc)    1
avg(cc)    1
dtype: int64


In [65]:
def calculate_weights(X_train, X_test):
  weights = []
  for i, row in X_train.iterrows():
      h = ((X_test.min() <= row) & (row <= X_test.max())).astype(int)
      si = h.sum()
      weight = 1 / ((len(row) - si + 1) ** 2)
      weights.append(weight)
  return np.array(weights)

In [66]:
calculate_weights(X_train, X_test)

array([1.        , 1.        , 1.        , ..., 0.25      , 0.11111111,
       1.        ])

In [ ]:
def ma_transfer_naive_bayes(X_train, X_test, y_train):
    def calculate_weights(X_train, X_test):
        weights = []
        for i, row in X_train.iterrows():
            h = ((X_test.min() <= row) & (row <= X_test.max())).astype(int)
            si = h.sum()
            weight = 1 / ((len(row) - si + 1) ** 2)
            weights.append(weight)
        return np.array(weights)

    weights = calculate_weights(X_train, X_test)
    clf = GaussianNB()
    clf.fit(X_train, y_train, sample_weight=weights)
    predictions = clf.predict_proba(X_test)[:, 1]

    return predictions